In [1]:
import sys

sys.path.append("../../")
import os
import pprint
import logging
import argparse

from shutil import copyfile

import yaml
import numpy as np
import torch
import torch.optim as optim

from accuracy import Accuracy, Accuracy3D
from models.stp_net import STPNet

from common.audio_processor import LibrosaAudioProcessor
from common.batcher.batch_processor import SimpleBatchProcessor
from common.datasets_parsers.av_parser import AVDBParser
from common.group_random_sampler import GroupRandomSampler
from common.images_batcher import ImagesBatcher
from common.logging.logging_control import create_logger
from common.losses import *
from common.lr_schedulers import *
from common.net_trainer import NetTrainer


def get_net_params(net, lr, weight_decay):
    # large lr for last fc parameters
    params = []
    for name, value in net.named_parameters():
        if not value.requires_grad:
            continue
        if "bias" in name:
            params += [{"name": name, "params": value, "lr": lr, "weight_decay": 0}]
        else:
            params += [
                {"name": name, "params": value, "lr": lr, "weight_decay": weight_decay}
            ]
    return params


def train():
    '''
    arguments_parser = argparse.ArgumentParser(
        formatter_class=argparse.ArgumentDefaultsHelpFormatter
    )
    arguments_parser.add_argument(
        "--config", help="yml config path", type=str, required=True
    )
    arguments_parser.add_argument(
        "--host", help="name of the host server", type=str, required=True
    )
    '''

    #args = arguments_parser.parse_args()

    # get host name for brevity
    #host = args.host
    host = "local"
    config_path = "A:/Профиль/Rab Table/Учёба/3/нейронные сети/4/pytorch/AEmotionNet/config.yml"

    # read configuration
    with open(config_path) as yml_file:
        cfg = yaml.load(Loader=yaml.FullLoader, stream=yml_file)
        params = {
            key: cfg[key]
            for key in cfg
            if not key in ["dataset", "ini_net", "logging", "test"]
        }
        dataset = {key: cfg["dataset"][key] for key in cfg["dataset"]}
        log = {key: cfg["logging"][key] for key in cfg["logging"]}
        ini_net = cfg["ini_net"]
    # init logging
    experiment_name = params["train"]["experiment_name"] + str(
        params["train"]["cuda_device"]
    )
    create_logger(
        log["log_dir"],
        experiment_name + ".log",
        console_level=logging.CRITICAL,
        file_level=logging.NOTSET,
    )

    # log configuration
    pp = pprint.PrettyPrinter(indent=4)
    logging.info("Configuration is:\n%s" % pp.pformat(cfg))
    experiment_path = os.path.join(log["log_dir"], params["train"]["experiment_name"])
    if not os.path.exists(experiment_path):
        os.makedirs(experiment_path)
    copyfile(config_path, os.path.join(experiment_path, "config.yml"))

    device = torch.device(
        "cuda:" + str(params["train"]["cuda_device"])
        if torch.cuda.is_available()
        else "cpu"
    )
    torch.manual_seed(params["seed"])
    torch.cuda.manual_seed_all(params["seed"])

    train_dataset_parser = AVDBParser(
        dataset_root=dataset["train"]["data_root"],
        file_list=dataset["train"]["file_list"],
        max_num_clips=params["parser"]["max_num_clips"],
        max_num_samples=params["parser"]["max_num_samples"],
        ungroup=False,
        normalize="AFEW-VA" in dataset["train"]["data_root"],
    )
    valid_dataset_parser = AVDBParser(
        dataset_root=dataset["valid"]["data_root"],
        file_list=dataset["valid"]["file_list"],
        max_num_clips=params["parser"]["max_num_clips"],
        max_num_samples=params["parser"]["max_num_samples"],
        ungroup=False,
        normalize="AFEW-VA" in dataset["train"]["data_root"],
    )

    softmax_size = (
        params["net"]["softmax_size"]
        if params["net"]["softmax_size"] > 0
        else train_dataset_parser.get_class_num()
    )
    print("\rsoftmax_size = %d" % softmax_size)

    # create train data sampler
    train_data_sampler = GroupRandomSampler(
        data=train_dataset_parser.get_data(),
        num_sample_per_classes=1,
        samples_is_randomize=True,
        step_size_for_samples=1,
    )
    valid_data_sampler = GroupRandomSampler(
        data=valid_dataset_parser.get_data(),
        num_sample_per_classes=1,
        samples_is_randomize=True,
        step_size_for_samples=1,
        is_shuffle=False,
    )

    # create train audio processor
    audio_processor = LibrosaAudioProcessor(
        type=params["preproc"]["audio"]["type"],
        sampling_rate=params["preproc"]["audio"]["sampling_rate"],
        max_wave_value=params["preproc"]["audio"]["max_wave_value"],
        sfft=params["preproc"]["audio"]["sfft"],
        mel=params["preproc"]["audio"]["mel"],
    )

    # create train image batcher
    train_images_batcher = ImagesBatcher(
        queue_size=params["train_batcher"]["queue_size"],
        batch_size=params["train_batcher"]["batch"],
        data_sampler=train_data_sampler,
        audio_processor=audio_processor,
        single_epoch=False,
        cache_data=False,
        disk_reader_process_num=params["train_batcher"]["disk_reader_process_num"],
    )
    valid_images_batcher = ImagesBatcher(
        queue_size=params["valid_batcher"]["queue_size"],
        batch_size=params["valid_batcher"]["batch"],
        data_sampler=valid_data_sampler,
        audio_processor=audio_processor,
        single_epoch=True,
        cache_data=False,
        disk_reader_process_num=params["valid_batcher"]["disk_reader_process_num"],
    )

    # create batch processor
    batch_processor = SimpleBatchProcessor(
        use_pin_memory=params["batch_proc"]["use_pin_memory"],
        cuda_id=params["train"]["cuda_device"],
        use_async=params["batch_proc"]["use_async"],
    )

    net_type = params["net"]["type"]
    if net_type == "STPNet":
        net = STPNet(softmax_size, depth=params["net"]["depth"])
    else:
        print("Type net is not supported!")
        exit(0)

    # net.apply(weights_init)
    loss = TotalLoss(params["losses"], 1, params["train"]["cuda_device"])

    net.to(device)
    loss.to(device)

    if params["net"]["fine_tune"]:
        net.load_state_dict(torch.load(ini_net))

    lr = params["opt"]["lr"]
    momentum = params["opt"]["momentum"]
    weight_decay = params["opt"]["weight_decay"]

    opt_type = params["opt"]["type"]
    if opt_type == "SGD":
        optimizer = optim.SGD(
            [{"params": net.parameters()}, {"params": loss.parameters()}],
            lr=lr,
            momentum=momentum,
            weight_decay=weight_decay,
            nesterov=True,
        )  # , caffe_like=True)
    if opt_type == "ASGD":
        optimizer = optim.ASGD(
            [{"params": net.parameters()}, {"params": loss.parameters()}],
            lr=lr,
            weight_decay=weight_decay,
        )  # , caffe_like=True)
    if opt_type == "Adagrad":
        optimizer = optim.Adagrad(
            [{"params": net.parameters()}, {"params": loss.parameters()}],
            lr=lr,
            weight_decay=weight_decay,
        )
    if opt_type == "Adadelta":
        optimizer = optim.Adadelta(
            [{"params": net.parameters()}, {"params": loss.parameters()}],
            lr=lr,
            weight_decay=weight_decay,
        )
    if opt_type == "Adam":
        optimizer = optim.Adam(
            [{"params": net.parameters()}, {"params": loss.parameters()}],
            lr=lr,
            weight_decay=weight_decay,
        )
    if opt_type == "RMSprop":
        optimizer = optim.RMSprop(
            [{"params": net.parameters()}, {"params": loss.parameters()}],
            lr=lr,
            weight_decay=weight_decay,
            momentum=momentum,
        )
    if opt_type == "LBFGS":
        optimizer = optim.LBFGS(
            net.parameters(),
            lr=lr,
            max_iter=5,
            max_eval=None,
            tolerance_grad=1e-05,
            tolerance_change=1e-09,
            history_size=1,
            line_search_fn=None,
        )

    # if params['net']['init'] == 'fine_tune':
    #    optimizer = cp['optimizer']

    lr_scheduler_type = params["lr_scheduler"]["type"]
    gamma = float(params["lr_scheduler"]["gamma"])
    if lr_scheduler_type == "ReduceLROnPlateau":
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=gamma)
    if lr_scheduler_type == "MultiStepLR":
        scheduler = torch.optim.lr_scheduler.MultiStepLR(
            optimizer, milestones=[15000, 30000, 60000], gamma=gamma
        )
    if lr_scheduler_type == "ExponentialLR":
        scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=gamma)
    if lr_scheduler_type == "SGDR":
        scheduler = SGDR_scheduler(
            optimizer,
            lr_start=lr,
            lr_end=gamma * lr,
            lr_period=params["train"]["epoch_size"] // params["train"]["step_size"],
            scale_lr=params["lr_scheduler"]["scale_lr"],
            scale_lr_fc=params["lr_scheduler"]["scale_lr_fc"],
        )

    net_trainer = NetTrainer(
        logs_dir=log["tb_log_dir"],
        cuda_id=params["train"]["cuda_device"],
        experiment_name=experiment_name,
        snapshot_dir=log["snapshot_dir"],
        config_data=cfg,
    )

    accuracy_fn = Accuracy3D(
        valid_dataset_parser.get_data(),
    )

    # lets try to train
    torch.set_num_threads(2)
    net_trainer.train(
        model=net,
        optimizer=optimizer,
        scheduler=scheduler,
        train_data_batcher=train_images_batcher,
        val_data_batcher=valid_images_batcher,
        val_iter=params["train"]["validate_iter"],
        batch_processor=batch_processor,
        loss_function=loss,
        max_iter=params["train"]["max_iter"],
        step_size=params["train"]["step_size"],
        snapshot_iter=params["train"]["snapshot_iter"],
        step_print=params["train"]["step_print"],
        accuracy_function=accuracy_fn,
    )

    # finalize data loader queues
    train_images_batcher.finish()
    valid_images_batcher.finish()


def test():
    '''
    arguments_parser = argparse.ArgumentParser(
        formatter_class=argparse.ArgumentDefaultsHelpFormatter
    )
    arguments_parser.add_argument(
        "--config", help="yml config path", type=str, required=True
    )
    arguments_parser.add_argument(
        "--host", help="name of the host server", type=str, required=True
    )
    '''

    #args = arguments_parser.parse_args()

    # get host name for brevity
    #host = args.host
    host = "local"
    config_path = "A:/Профиль/Rab Table/Учёба/3/нейронные сети/4/pytorch/AEmotionNet/config.yml"

    # read configuration
    with open(config_path) as yml_file:
        cfg = yaml.load(Loader=yaml.FullLoader, stream=yml_file)
        train_params = {
            key: cfg[key]
            for key in cfg
            if not key in ["dataset", "ini_net", "logging", "test"]
        }
        test_params = cfg["test"]
        dataset = {
            key: test_params["dataset"][key] for key in test_params["dataset"]
        }

    device = torch.device(
        "cuda:" + str(test_params["cuda_device"]) if torch.cuda.is_available() else "cpu"
    )
    torch.manual_seed(train_params["seed"])
    torch.cuda.manual_seed_all(train_params["seed"])
    cp = torch.load(test_params["file_model"])

    test_dataset_parser = AVDBParser(
        dataset_root=dataset["data_root"],
        file_list=dataset["test_file_list"],
        ungroup=train_params["preproc"]["data_frame"]["depth"] == 1,
    )

    test_data_sampler = GroupRandomSampler(
        data=test_dataset_parser.get_data(),
        num_sample_per_classes=train_params["preproc"]["data_frame"]["depth"],
        samples_is_randomize=train_params["sampler"]["samples_is_randomize"],
        step_size_for_samples=train_params["sampler"]["step_size_for_samples"],
        is_shuffle=False,
    )

    test_audio_processor = LibrosaAudioProcessor(
        type=train_params["preproc"]["audio"]["type"],
        sampling_rate=train_params["preproc"]["audio"]["sampling_rate"],
        max_wave_value=train_params["preproc"]["audio"]["max_wave_value"],
        sfft=train_params["preproc"]["audio"]["sfft"],
        mel=train_params["preproc"]["audio"]["mel"],
    )

    test_images_batcher = ImagesBatcher(
        queue_size=train_params["valid_batcher"]["queue_size"],
        batch_size=train_params["valid_batcher"]["batch"],
        data_sampler=test_data_sampler,
        audio_processor=test_audio_processor,
        single_epoch=True,
        cache_data=False,
        disk_reader_process_num=train_params["valid_batcher"]["disk_reader_process_num"],
    )

    batch_processor = SimpleBatchProcessor(
        use_pin_memory=train_params["batch_proc"]["use_pin_memory"],
        cuda_id=train_params["train"]["cuda_device"],
        use_async=train_params["batch_proc"]["use_async"],
    )

    model = cp["model"]
    model.to(device)
    model.eval()

    test_images_batcher.start()
    predict = np.zeros((0, 2), dtype=np.float32)
    while True:
        batch = test_images_batcher.next_batch()
        if batch is None:
            break

        data, _, _ = batch_processor.pre_processing(batch)
        logits = model(data)
        predict = np.concatenate((predict, logits.cpu().data.numpy()), axis=0)

    accuracy_fn = Accuracy(test_dataset_parser.get_data())
    accuracy_fn.by_frames(predict)
    accuracy_fn.by_clips(predict)


if __name__ == "__main__":
    train()
    # test()


AVDB meta parsing: 100%|██████████| 42956/42956 [00:02<00:00, 17381.51images/s]


softmax_size = 2
disk_reader_process_num: 1
disk_reader_process_num: 1


2023-12-13 18:48:03,448:CRITICAL:EmoV2_step40: iteration: 0: Loss: 0.602749, lr: 0.009000
2023-12-13 18:48:07,264:CRITICAL:EmoV2_step40: validate. Iteration: 0: Accuracy (valence, arousal): 15.044% 3.097%
2023-12-13 18:48:07,266:CRITICAL:EmoV2_step40: validate. Iteration: 0: Loss: 0.332374
2023-12-13 18:48:07,411:CRITICAL:EmoV2_step40: iteration: 10: Loss: 0.265822, lr: 0.009000


targets = (226, 2), predict = (226, 2)
Valence:
   accuracy per clips: 15.044%
   error per clips: mean=0.287, std=0.201
Arousal:
   accuracy per clips: 3.097%
   error per clips: mean=0.547, std=0.318
---------



2023-12-13 18:48:07,784:CRITICAL:EmoV2_step40: iteration: 20: Loss: 0.227244, lr: 0.009000
2023-12-13 18:48:08,109:CRITICAL:EmoV2_step40: iteration: 30: Loss: 0.042331, lr: 0.009000
2023-12-13 18:48:08,433:CRITICAL:EmoV2_step40: iteration: 40: Loss: 0.290871, lr: 0.009000
2023-12-13 18:48:08,679:CRITICAL:EmoV2_step40: iteration: 50: Loss: 0.126779, lr: 0.009000
2023-12-13 18:48:08,958:CRITICAL:EmoV2_step40: iteration: 60: Loss: 0.042577, lr: 0.009000
2023-12-13 18:48:09,243:CRITICAL:EmoV2_step40: iteration: 70: Loss: 0.116371, lr: 0.009000
2023-12-13 18:48:09,589:CRITICAL:EmoV2_step40: iteration: 80: Loss: 0.194849, lr: 0.009000
2023-12-13 18:48:09,892:CRITICAL:EmoV2_step40: iteration: 90: Loss: 0.120074, lr: 0.009000
2023-12-13 18:48:10,242:CRITICAL:EmoV2_step40: iteration: 100: Loss: 0.119287, lr: 0.009000
2023-12-13 18:48:14,129:CRITICAL:EmoV2_step40: validate. Iteration: 100: Accuracy (valence, arousal): 16.814% 23.894%
2023-12-13 18:48:14,131:CRITICAL:EmoV2_step40: validate. Itera

...

targets = (226, 2), predict = (226, 2)
Valence:
   accuracy per clips: 22.124%
   error per clips: mean=0.176, std=0.123
Arousal:
   accuracy per clips: 18.142%
   error per clips: mean=0.266, std=0.228
---------



2023-12-13 18:53:50,739:CRITICAL:EmoV2_step40: iteration: 4820: Loss: 0.127573, lr: 0.008799
2023-12-13 18:53:51,088:CRITICAL:EmoV2_step40: iteration: 4830: Loss: 0.084569, lr: 0.008797
2023-12-13 18:53:51,439:CRITICAL:EmoV2_step40: iteration: 4840: Loss: 0.068837, lr: 0.008797
2023-12-13 18:53:51,788:CRITICAL:EmoV2_step40: iteration: 4850: Loss: 0.167662, lr: 0.008795
2023-12-13 18:53:52,106:CRITICAL:EmoV2_step40: iteration: 4860: Loss: 0.061691, lr: 0.008795
2023-12-13 18:53:52,461:CRITICAL:EmoV2_step40: iteration: 4870: Loss: 0.158706, lr: 0.008795
2023-12-13 18:53:52,823:CRITICAL:EmoV2_step40: iteration: 4880: Loss: 0.133688, lr: 0.008793
2023-12-13 18:53:53,257:CRITICAL:EmoV2_step40: iteration: 4890: Loss: 0.217376, lr: 0.008793
2023-12-13 18:53:53,581:CRITICAL:EmoV2_step40: iteration: 4900: Loss: 0.094775, lr: 0.008791
2023-12-13 18:53:57,471:CRITICAL:EmoV2_step40: validate. Iteration: 4900: Accuracy (valence, arousal): 24.336% 23.451%
2023-12-13 18:53:57,473:CRITICAL:EmoV2_step4

targets = (226, 2), predict = (226, 2)
Valence:
   accuracy per clips: 24.336%
   error per clips: mean=0.218, std=0.144
Arousal:
   accuracy per clips: 23.451%
   error per clips: mean=0.253, std=0.211
---------



2023-12-13 18:53:58,044:CRITICAL:EmoV2_step40: iteration: 4920: Loss: 0.295822, lr: 0.008791
2023-12-13 18:53:58,340:CRITICAL:EmoV2_step40: iteration: 4930: Loss: 0.052701, lr: 0.008788
2023-12-13 18:53:58,627:CRITICAL:EmoV2_step40: iteration: 4940: Loss: 0.146004, lr: 0.008788
2023-12-13 18:53:58,954:CRITICAL:EmoV2_step40: iteration: 4950: Loss: 0.049436, lr: 0.008786
2023-12-13 18:53:59,314:CRITICAL:EmoV2_step40: iteration: 4960: Loss: 0.147382, lr: 0.008786
2023-12-13 18:53:59,639:CRITICAL:EmoV2_step40: iteration: 4970: Loss: 0.030629, lr: 0.008786
2023-12-13 18:54:00,020:CRITICAL:EmoV2_step40: iteration: 4980: Loss: 0.022388, lr: 0.008784
2023-12-13 18:54:00,311:CRITICAL:EmoV2_step40: iteration: 4990: Loss: 0.057063, lr: 0.008784
2023-12-13 18:54:00,641:CRITICAL:EmoV2_step40: iteration: 5000: Loss: 0.102628, lr: 0.008782
2023-12-13 18:54:04,526:CRITICAL:EmoV2_step40: validate. Iteration: 5000: Accuracy (valence, arousal): 23.894% 23.451%
2023-12-13 18:54:04,527:CRITICAL:EmoV2_step4

targets = (226, 2), predict = (226, 2)
Valence:
   accuracy per clips: 23.894%
   error per clips: mean=0.236, std=0.155
Arousal:
   accuracy per clips: 23.451%
   error per clips: mean=0.246, std=0.209
---------

